Input necessary classes

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os

Read input data


In [188]:
input_data = pd.read_excel('Input.xlsx')


In [189]:
input_data

,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...
...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...
97,blackassign0098,https://insights.blackcoffer.com/contribution-...
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...


Function to extract article text from URL and save as text file






In [190]:
#Function to extract article text from URL and save as text file

def extract_and_save_article(url, url_id):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Extract article title
    title = soup.find('title').get_text()
    # Find the main content area based on specific attributes or structure
    main_content = soup.find('div', class_='td-post-content tagdiv-type')  # Assuming article content is within this class
    if not main_content:
        main_content = soup.find('div', class_='tdb-block-inner td-fix-index')  # If not found, try finding in this class
    if not main_content:
        main_content = soup.body  # If main content not found, use entire body

    # Extract article text from main content
    article_text = ""
    list_items = main_content.find_all('li')
    if list_items:
        # If list items exist, include paragraphs and list items in extraction
        for element in main_content.find_all(['p', 'li']):
            if element.name == 'p':  # Check if current element is a paragraph
                article_text += element.get_text() + "\n"
            elif element.name == 'li':  # Check if current element is a list item
                article_text += element.get_text() + "\n"
    else:
        # If no list items, only include paragraphs in extraction
        for paragraph in main_content.find_all('p'):
            article_text += paragraph.get_text() + "\n"


    # Create directory if not exists
    output_dir = "Output Extract"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Save article text to a text file
    with open(os.path.join(output_dir, f"{url_id}.txt"), "w", encoding="utf-8") as file:
        file.write(title + "\n")
        file.write(article_text)



 Extract article text for each URL in the input data


In [191]:

# Extract article text for each URL in the input data

for index, row in input_data.iterrows():
    extract_and_save_article(row['URL'], row['URL_ID'])



#Data Analysis


Import Classes

In [192]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import re
import string


In [193]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

 Load stopwords


In [194]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#1. Sentimental Analysis

Clean Stopwords

In [195]:

def load_stopwords(stopwords_folder):
    stopwords = {}
    for filename in os.listdir(stopwords_folder):
        if filename.startswith("StopWords_") and filename.endswith(".txt"):
            category = filename.split("_")[1].split(".")[0]  # Extract category from filename
            with open(os.path.join(stopwords_folder, filename), "r", encoding="utf-8",errors="ignore") as file:
                stopwords[category] = set(file.read().splitlines())
    return stopwords

def clean_text(text, stopwords):
    cleaned_text = " ".join(word for word in text.split() if word.lower() not in stopwords)
    return cleaned_text

def clean_all_files(input_dir, output_dir, stopwords_folder):
    # Create output directory if not exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Load all stop words files
    stopwords = load_stopwords(stopwords_folder)

    # Iterate through each text file in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith(".txt"):
            file_path = os.path.join(input_dir, filename)
            output_file_path = os.path.join(output_dir, filename)

            # Read text from input file
            with open(file_path, "r", encoding="utf-8") as file:
                text = file.read()

            # Clean text by removing stop words for each category
            cleaned_text = text
            for category, words in stopwords.items():
                cleaned_text = clean_text(cleaned_text, words)

            # Save cleaned text to output file
            with open(output_file_path, "w", encoding="utf-8") as file:
                file.write(cleaned_text)


In [196]:
# Specify input directory containing 100 text files
input_directory = "Output Extract"

# Specify output directory to save cleaned text files
output_directory = "Output Clean"

# Specify folder containing stop words files
stopwords_folder = "StopWords"

# Clean all text files
clean_all_files(input_directory, output_directory, stopwords_folder)

 Load positive and negative words



In [197]:
# Function to load words from file using different encodings
def load_words_from_file(filename):
    for encoding in encodings_to_try:
        try:
            with open(filename, 'r', encoding=encoding) as file:
                return [line.strip().lower() for line in file]
        except UnicodeDecodeError:
            continue
    # Raise an error if none of the encodings work
    raise UnicodeDecodeError(f"Unable to decode file '{filename}' using any of the specified encodings: {encodings_to_try}")

In [198]:
positive_words = set()
negative_words = set()

# Define a list of possible encodings to try
encodings_to_try = ['utf-8', 'latin-1']  # Add more encodings if needed


# Function to load words from file using different encodings
def load_words_from_file(filename):
    for encoding in encodings_to_try:
        try:
            with open(filename, 'r', encoding=encoding) as file:
                return [line.strip().lower() for line in file]
        except UnicodeDecodeError:
            continue
    # Raise an error if none of the encodings work
    raise UnicodeDecodeError(f"Unable to decode file '{filename}' using any of the specified encodings: {encodings_to_try}")

positive_words = set(load_words_from_file("MasterDictionary/positive-words.txt"))
negative_words = set(load_words_from_file("MasterDictionary/negative-words.txt"))

In [199]:
def calculate_derived_variables(text, positive_words, negative_words):
    # Tokenize the text into words
    words = word_tokenize(text)

    # Initialize variables for positive and negative scores
    pos_score = 0
    neg_score = 0

    # Count positive and negative words
    for word in words:
        if word.lower() in positive_words:
            pos_score += 1
        elif word.lower() in negative_words:
            neg_score += 1

    # Calculate polarity score
    polarity_score = (pos_score - neg_score) / (pos_score + neg_score + 0.000001)

    # Calculate subjectivity score
    total_words = len(words)
    subjectivity_score = (pos_score + neg_score) / (total_words + 0.000001)

    return pos_score, neg_score, polarity_score, subjectivity_score

#2.	Analysis of Readability

In [200]:
# Function to calculate readability analysis variables

def calculate_readability(text):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)

    # Tokenize the text into words
    words = [word.lower() for word in word_tokenize(text) if word.isalnum()]

    # Remove punctuation
    words = [word for word in words if word not in stop_words and word not in string.punctuation]

    # Initialize variables for readability analysis
    total_words = len(words)
    total_sentences = len(sentences)
    complex_word_count = 0

    # Calculate the average sentence length
    average_sentence_length = total_words / total_sentences

    # Calculate the percentage of complex words
    for word in words:
        if syllable_count(word) > 2:  # Assuming words with more than 2 syllables are complex
            complex_word_count += 1

    percentage_complex_words = (complex_word_count / total_words) * 100

    # Calculate Fog Index
    fog_index = 0.4 * (average_sentence_length + percentage_complex_words)

    return average_sentence_length, percentage_complex_words, fog_index, complex_word_count

# Function to count syllables in a word
def syllable_count(word):
    vowels = 'aeiouy'
    count = 0
    prev_char_is_vowel = False
    for char in word:
        char = char.lower()
        if char in vowels and not prev_char_is_vowel:
            count += 1
            prev_char_is_vowel = True
        elif char not in vowels:
            prev_char_is_vowel = False
    if len(word) > 2 and word.endswith('e') and not word.endswith('le'):
        count -= 1
    return max(count, 1)

#3. Average Number of Words Per Sentence

In [201]:
# Function to calculate Average Number of Words Per Sentence variables

def average_words_per_sentence(text):
    # Tokenize the text into sentences
    sentences = nltk.sent_tokenize(text)

    # Count the total number of words
    words = nltk.word_tokenize(text)
    total_words = len(words)

    # Count the total number of sentences
    total_sentences = len(sentences)

    # Calculate the average number of words per sentence
    if total_sentences > 0:
        avg_words_per_sentence = total_words / total_sentences
    else:
        avg_words_per_sentence = 0

    return avg_words_per_sentence

# 4. 	Word Count

In [202]:
#Function to calculate Word Count variables
def count_cleaned_words(text):
    # Tokenize the text into words
    words = nltk.word_tokenize(text)

    # Remove punctuation
    words = [word for word in words if word not in string.punctuation]

    # Count the cleaned words
    word_count = len(words)

    return word_count

#5. Syllable Count Per Word

In [203]:
# Function to Calculate Syllable Count Per Word variables

def count_syllables_per_text(text):
    total_syllables = 0
    vowels = 'aeiouy'

    # Tokenize the text into words
    words = [word.lower() for word in word_tokenize(text) if word.isalnum()]

    # Remove punctuation
    words = [word for word in words if word not in stop_words and word not in string.punctuation]

    # Count syllables for each word
    for word in words:
        count = 0
        prev_char_is_vowel = False

        # Handle exceptions for words ending with "es" or "ed"
        if word.endswith('es') or word.endswith('ed'):
            total_syllables += 0
            continue

        for char in word:
            char = char.lower()
            if char in vowels and not prev_char_is_vowel:
                count += 1
                prev_char_is_vowel = True
            elif char not in vowels:
                prev_char_is_vowel = False

        # Apply additional rules or exceptions as needed

        total_syllables += max(count, 1)  # Ensure minimum syllable count of 1

    return total_syllables


#6.	Personal Pronouns

In [204]:
# Function to Calculate Personal Pronouns variables

def count_personal_pronouns(text):
    # Define the list of personal pronouns
    personal_pronouns = ["I", "we", "my", "ours", "us"]

    # Compile the regex pattern to match the personal pronouns
    pattern = re.compile(r'\b(?:' + '|'.join(personal_pronouns) + r')\b', flags=re.IGNORECASE)

    # Find all matches of personal pronouns in the text
    matches = re.findall(pattern, text)

    # Count the total occurrences of personal pronouns
    personal_pronouns_count = len(matches)

    return personal_pronouns_count


#7. Average Word Length

In [205]:
def calculate_average_word_length(text):
    # Tokenize the text into words
    words = text.split()

    # Calculate the total number of characters in all words
    total_characters = sum(len(word) for word in words)

    # Count the total number of words
    total_words = len(words)

    # Calculate the average word length
    if total_words > 0:
        average_word_length = total_characters / total_words
    else:
        average_word_length = 0

    return average_word_length


# Result

In [206]:
results = []

for filename in os.listdir(output_directory):
    if filename.endswith('.txt'):
        # Read the text from the file
        with open(os.path.join(output_directory, filename), 'r', encoding='utf-8') as file:
            text = file.read()

        # Calculate derived variables
        pos_score, neg_score, polarity_score, subjectivity_score = calculate_derived_variables(text, positive_words, negative_words)

        # Calculate readability analysis variables
        average_sentence_length, percentage_complex_words, fog_index, complex_word_count = calculate_readability(text)

        # Calculate Average Number of Words Per Sentence variables
        avg_words_per_sentence = average_words_per_sentence(text)

        # Calculate Word Count variables
        word_count = count_cleaned_words(text)

        # Calculate the Syllable Count Per Word variable
        total_syllables = count_syllables_per_text(text)
        avg_syllables_per_word = total_syllables / word_count if word_count > 0 else 0

        # Calculate Personal Pronouns Count variables
        pronoun_count = count_personal_pronouns(text)

        # Calculate Word Count variables
        avg_word_length = calculate_average_word_length(text)

        # Extract URL ID from filename
        url_id = os.path.splitext(filename)[0]

        # Append results to the list
        results.append({
            "URL_ID": url_id,
            "Positive Score": pos_score,
            "Negative Score": neg_score,
            "Polarity Score": polarity_score,
            "Subjectivity Score": subjectivity_score,
            "AVG SENTENCE LENGTH": average_sentence_length,
            "PERCENTAGE OF COMPLEX WORDS": percentage_complex_words,
            "FOG INDEX": fog_index,
            "AVG NUMBER OF WORDS PER SENTENCE": avg_words_per_sentence,
            "COMPLEX WORD COUNT": complex_word_count,
            "WORD COUNT": word_count,
            "SYLLABLE PER WORD": avg_syllables_per_word,
            "Personal Pronouns Count": pronoun_count,
            "Average Word Length": avg_word_length
        })

# Create DataFrame from results
output_df = pd.DataFrame(results)
# Sort the results based on the URL ID
output_df = output_df.sort_values(by="URL_ID")
# Print the DataFrame
print(output_df)

             URL_ID  Positive Score  Negative Score  Polarity Score  \
0   blackassign0001              33               6        0.692308   
1   blackassign0002              60              31        0.318681   
2   blackassign0003              38              24        0.225806   
3   blackassign0004              38              75       -0.327434   
4   blackassign0005              21               8        0.448276   
..              ...             ...             ...             ...   
95  blackassign0096              29              57       -0.325581   
96  blackassign0097              25              35       -0.166667   
97  blackassign0098               7               2        0.555555   
98  blackassign0099               0               0        0.000000   
99  blackassign0100               1               0        0.999999   

    Subjectivity Score  AVG SENTENCE LENGTH  PERCENTAGE OF COMPLEX WORDS  \
0             0.053793             7.466667                    28.03571

In [207]:
# Merge the two dataframes based on the URL ID column
merged_df = pd.merge(output_df, input_data[['URL_ID', 'URL']], on='URL_ID')

# Reorganize the columns
merged_df = merged_df[['URL_ID', 'URL', 'Positive Score', 'Negative Score', 'Polarity Score',
                       'Subjectivity Score', 'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS',
                       'FOG INDEX', 'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT',
                       'WORD COUNT', 'SYLLABLE PER WORD', 'Personal Pronouns Count', 'Average Word Length']]


In [208]:

# Save the merged dataframe to a new Excel file
merged_df.to_excel("Output Data Structure.xlsx", index=False)